## Task 1

Ознайомитися із API Національного банку  
    "https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange\
> 1. Підключитися до API.
> 2. Отримати курс долара за останній рік.
> 3. Вивести середнє значення та відхилення курсу за кожний місяць.
> 4. Дану інформацію записати у файл за допомогою pickle.

In [35]:
from datetime import timedelta, date
import requests


start = date(2020, 7, 1)
end = date(2021, 7, 1)


def date_range(start, end):
    """Генераторна функція, яка обчислює кількість днів у заданому
    діапазоні дат та потім переводить кожен день у формат дати.
    Використовується у функції "d_period"
    
    """
    for day in range(int((end - start).days)):
        yield start + timedelta(day)


def d_period():
    """Формує список дат у форматі, заданому в API.
    
    """
    period = []
    for day in date_range(start, end):
        period.append(day.strftime("%Y%m%d"))
    return period
        

def pars_usd():
    """Збирає дані з сайту по заданому періоду дат.
    Створює список словників зі всіма даними, які дає розділ до якого звертаємось.
    Кожен словник - це набір даних на одну дату.
    
    """
    db = []
    for date in d_period():
        site = f'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?valcode=USD&date={date}&json' 
        r = requests.get(site)

        if r.status_code == 200:
            data = r.json()
            db.append(data)
        else:
            print("Wrong link or site does not respond...")
            break
    return db


def rate_date():
    """Створює словник, де ключами є дати, а значеннями - курс долара.
    
    """
    clean_dict = {}
    for x in pars_usd():
        for y in x:
            clean_dict[y['exchangedate']] = y['rate'] 
    return clean_dict
dict_of_rates = rate_date()


def month_rate(rate_date):
    """Прибирає день із словника та перетворює його на список кортежів, 
    де першим значенням є 'місяць.рік', а другим - курс на дату.
    
    """
    clean_from_day = []
    for k, v in dict_of_rates.items():
        clean_from_day.append((k[3:], v))
    return clean_from_day


def month_dict(month_rate):
    """Ітерується по датам в 'rate_day_clean' і створює словник, де
    ключем є 'місяць.рік', а значенням - список курсів за місяць.
    
    """
    prev_date = ''
    month_rates = dict()
    tmp_list = None 
    for date, rate in month_rate(rate_date):
        if prev_date != date:
            tmp_list = []
            month_rates[date] = tmp_list
            prev_date = date
        tmp_list.append(rate)
    return month_rates
final_rates = month_dict(month_rate)


def rate_calc(month_rate):
    """Розраховує середній курс за кожен місяць.
    Розраховує індекс відхилення курсу до попереднього місяця.
    Друкує результати обчислень.
    
    """
    everage_rate = []
    for date, rates in final_rates.items():
        everage_rate.append((date, (round(sum(rates)/len(rates),2))))
    
    change_per_m = []
    for i in range(len(everage_rate)-1):
        change_per_m.append(round((everage_rate[i+1][1]/everage_rate[i][1])*100,2))
    change_per_m.insert(0, '-')
        
    l_everage_rate = []
    for i in everage_rate:
        l_everage_rate.append(list(i))
    
    result = list(zip(l_everage_rate, change_per_m))
    return result

res = rate_calc(month_dict)

for i in res:
    print(f'Середньомісячний курс USD/UAH у {i[0][0]} становив {i[0][1]}\n\tкоефіціент зміни курсу за місяць: {i[1]}')

    
import pickle


res_for_pickle = rate_calc(month_rate)
f = open('curs_db.pickle', 'wb')
pickle.dump(res_for_pickle, f)
f.close()
print("\nФайл у форматі pickle успішно створено і збережено")

Середньомісячний курс USD/UAH у 07.2020 становив 27.31
	коефіціент зміни курсу за місяць: -
Середньомісячний курс USD/UAH у 08.2020 становив 27.52
	коефіціент зміни курсу за місяць: 100.77
Середньомісячний курс USD/UAH у 09.2020 становив 27.98
	коефіціент зміни курсу за місяць: 101.67
Середньомісячний курс USD/UAH у 10.2020 становив 28.32
	коефіціент зміни курсу за місяць: 101.22
Середньомісячний курс USD/UAH у 11.2020 становив 28.31
	коефіціент зміни курсу за місяць: 99.96
Середньомісячний курс USD/UAH у 12.2020 становив 28.17
	коефіціент зміни курсу за місяць: 99.51
Середньомісячний курс USD/UAH у 01.2021 становив 28.22
	коефіціент зміни курсу за місяць: 100.18
Середньомісячний курс USD/UAH у 02.2021 становив 27.88
	коефіціент зміни курсу за місяць: 98.8
Середньомісячний курс USD/UAH у 03.2021 становив 27.8
	коефіціент зміни курсу за місяць: 99.71
Середньомісячний курс USD/UAH у 04.2021 становив 27.93
	коефіціент зміни курсу за місяць: 100.47
Середньомісячний курс USD/UAH у 05.2021 с

### Summary for Task 1

In [1]:
import pickle


f = open('curs_db.pickle', 'rb')
new_curs = pickle.load(f)
f.close()
print("У файлі curs_db.pickle збережена наступна інформація:\n")
for i in new_curs:
    print(i)

У файлі curs_db.pickle збережена наступна інформація:

(['07.2020', 27.31], '-')
(['08.2020', 27.52], 100.77)
(['09.2020', 27.98], 101.67)
(['10.2020', 28.32], 101.22)
(['11.2020', 28.31], 99.96)
(['12.2020', 28.17], 99.51)
(['01.2021', 28.22], 100.18)
(['02.2021', 27.88], 98.8)
(['03.2021', 27.8], 99.71)
(['04.2021', 27.93], 100.47)
(['05.2021', 27.6], 98.82)
(['06.2021', 27.24], 98.7)


## Task 2
 
### Потрібно проаналізувати всі товари на сайті:  
    https://smallpacking.agrosem.ua/shop/
>Створіть базу даних, що містить таблицю із такими стовбцями:   
_- артикул товару_  
_- найменування_  
_- ціна, грн_  
_- вага_* (ваги на сайті немає, тому замість неї упаковка):

In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3


agrosem_link = "https://smallpacking.agrosem.ua/shop/"


def is_num(text):
    """Функція, яка слідкує за тим, аби в номері сторінки, на яку ми перейшли, було число.
    Викликається всередині функції 'get_next_page'.
    
    """
    try:
        int(text)
        return True
    except ValueError:
        return False
    
    
def basic_request(url):
    """Створює об'єкт soup для функцій:
    - 'get_next_page'
    - 'parse_product'
    - 'get_products_list'
                
    """
    response = requests.get(url)
    soup_result = BeautifulSoup(response.text, 'lxml')
    return soup_result


def get_next_page(current_page_link):
    """Додає +1 до поточної сторінки (переміщує курсор). 
    
    """
    soup = basic_request(current_page_link)
    page_numbers = soup.find('ul', class_='page-numbers')
    current_page = int(page_numbers.find('span', class_='page-numbers current').text)
    for page in page_numbers.find_all('a', class_='page-numbers'):
        if is_num(page.text):
            if int(page.text) == current_page + 1:
                next_page_link = page.get('href').strip('/')
                return next_page_link
    return None


def get_pages():
    """В парі із функцією 'get_next_page' створює список сторінок з переліком продуктів. 
    
    """
    pages_list = []
    current_page = agrosem_link
    while current_page is not None:
        pages_list.append(current_page)
        current_page = get_next_page(current_page)
    return pages_list


def parse_product(product_link):
    """Шукає інформацію по кожному продукту по заданим параметрам.
    
    """
    soup = basic_request(product_link)
    attributes_description = soup.find('div', class_='atributes-description')
    name = attributes_description.h2.text
    box_type = attributes_description.find('span', class_='type-packaging-name').text
    sku = attributes_description.find('span', class_='sku').text[5:]
    price = attributes_description.find('span', class_='regular-price').text.replace(' ', '')[:-3]
    return sku, name, box_type, price


def get_products_list():
    """Бере список сторінок у 'get_pages' та створює ссилки на продукти по черзі,
    передає їх у функцію parse_product, яка шукає потрібну інформацію по кожному продукту
    і повертає результат знову сюди у змінну products_list.
    
    """
    pages_list = get_pages()
    products_list = []
    for page in pages_list:
        soup = basic_request(page)
        products = soup.find('div', class_='product-category-wrapper')

        for product in products.find_all('div', class_='product-from-category-container'):
            description = product.a.get('href').strip('/')
            product_info = parse_product(description)
            products_list.append(product_info)
    return products_list

    
def create_table():
    connection = sqlite3.connect("Agro.db")
    crsr = connection.cursor()
    crsr.execute("""CREATE TABLE IF NOT EXISTS Agro(
    sku INT PRIMARY KEY,
    product TEXT,
    package TEXT,
    price REAL);""")
    connection.commit()
    crsr.close()
    connection.close()


def sqlite_rec(res):
    connection = sqlite3.connect("Agro.db")
    crsr = connection.cursor()
    crsr.executemany("INSERT OR REPLACE INTO Agro VALUES(?, ?, ?, ?);", res)
    connection.commit()
    crsr.close()
    connection.close()

    
result = get_products_list()
create_table()
sqlite_rec(result)
print('The database has been successfully created and filled with data')

The database has been successfully created and filled with data


### Summary for Task 2

In [4]:
import sqlite3


connection = sqlite3.connect("Agro.db")
crsr = connection.cursor()
crsr.execute("SELECT * FROM Agro;")
records = crsr.fetchall()
print("Всьго на сайті https://smallpacking.agrosem.ua/shop/ представлено:", len(records), 'товарів')


list_prices = []
for record in records:
    list_prices.append(record[3])
    average = round(sum(list_prices)/len(list_prices))
    maximum = round(max(list_prices))
    minimum = round(min(list_prices))
print(f'\nСередня ціна за товар {average} грн\nМаксимальна ціна за товар {maximum} грн\nМінімальна ціна за товар {minimum} грн')


crsr.execute("SELECT * FROM Agro;")
first_5 = crsr.fetchmany(5)
print(f'\nПерші 5 рядків створеної бази даних:\n')
for record in first_5:
    print(f'{record}')
    
connection.commit()
connection.close()

Всьго на сайті https://smallpacking.agrosem.ua/shop/ представлено: 119 товарів

Середня ціна за товар 6535 грн
Максимальна ціна за товар 56000 грн
Мінімальна ціна за товар 125 грн

Перші 5 рядків створеної бази даних:

(28164, 'Ferticare Hydro 6-14-30 + micro', 'Коробка', 1900.0)
(66353, 'Ferticare Kombi 14-11-25 + micro', 'Коробка', 1700.0)
(39566, 'MagPlon K (нітрат калію)', 'Коробка', 1200.0)
(69156, 'Pulan 34.4', 'Коробка', 7500.0)
(74266, 'Saletrosan®26', 'Коробка', 4400.0)


## Task 3

### Ознайомтеся із роботою SQLite та відповідним модулем у Python.
> Завантажте базу даних для виконання лабораторної роботи.  
> _**Підключіться до завантаженої бази SQLite.**_ 
>> 1. Виведіть інформацію про дану базу.
>> 2. Виведіть перелік всіх таблиць.
>> 3. Список всіх cust_id з таблиці Customers table.
>> 4. Всю таблицю Customers table
>> 5. Список клієнтів (cust_names) відсортованих від Z до A
>> 6. Таблицю клієнтів та замовлень (cust_id і order_num). Відсортуйте по клієнту і потім по даті замовлення
>> 7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості
>> 8. Товар (з таблиці Products), ціна якого становить 9.49
>> 9. Виведіть назву товару та ціну, яка лежить в діапазоні від 3 до 6. Відсортуйте результат в по ціні в порядку зростання
>> 10. Кількість товару, що було продано
>> 11. Кількість найменувань товару, ціна якого більша за 4
>> 12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі
>> 13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити
>> 14. Виведіть список клієнтів та їх замовлення (використайте JOIN)
>> 15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна  
Використайте UNION для об'єднання цих запитів"

In [2]:
import sqlite3


tysql = '/Users/Home/Desktop/Python/MainAcademy/tysql.sqlite' 
connection = sqlite3.connect(tysql)
crsr = connection.cursor()

    
crsr.execute('SELECT sqlite_version()')
vers_tysql = crsr.fetchone()
print(f'1. Версія бази даних SQLite: {vers_tysql}')

crsr.execute("SELECT count(*) from sqlite_master as tables WHERE type='table'")
q_tables = crsr.fetchone()
print(f'   База налічує {q_tables} таблиць.')

crsr.execute("SELECT name from sqlite_master as tables WHERE type='table'")
name_tables = crsr.fetchall()
print(f'\n2. Назви таблиць наступні:\n{name_tables}')

crsr.execute('SELECT cust_id FROM Customers')
cust_id = crsr.fetchall()
print(f'\n3. Список cust_id із таблиці Customers:\n{cust_id}')

crsr.execute('SELECT * FROM Customers')
all_cust = crsr.fetchall()
print(f'\n4. Таблиця Customers виглядає наступним чином:')
for i in all_cust:
    print(i)

crsr.execute('SELECT cust_name FROM Customers ORDER BY cust_name DESC')
cust_names = crsr.fetchall()
print('\n5. Список клієнтів, відсортованих від Z до A:')
for i in cust_names:
    print(i)

crsr.execute("""SELECT cust_name, Orders.cust_id, order_num, order_date
                FROM Orders 
                LEFT JOIN Customers 
                WHERE Orders.cust_id == Customers.cust_id
                ORDER BY cust_name, order_date""")
client_order = crsr.fetchall()
print(f'\n6. Таблиця, відсортована по клієнту і даті замовлення:\n   клієнт, id клієнта, номер, дата замовлення')
for i in client_order:
    print(i)

crsr.execute("""SELECT quantity, item_price
                FROM OrderItems 
                ORDER BY quantity DESC, item_price DESC""")
quan_price = crsr.fetchall()
print(f"""\n7. Таблиця OrderItems: кількість, вартість
   відсортована в порядку спадання по кількості та вартості""")
for i in quan_price:
    print(i)

crsr.execute('SELECT prod_name, prod_price FROM Products WHERE prod_price == 9.49')
product_name = crsr.fetchall()
print(f'\n8.Товари, ціна яких становить 9.49:')
for i in product_name:
    print(i)

crsr.execute("""SELECT prod_name, prod_price FROM Products 
                WHERE prod_price BETWEEN 3 AND 6
                ORDER BY prod_price""")
name_price_3_6 = crsr.fetchall()
print(f"""\n9. Назва і ціна товарів, які коштують вдіапазоні від 3 до 6
   відсортовані по ціні в порядку зростання""")
for i in name_price_3_6:
    print(i)

crsr.execute("SELECT sum(quantity) FROM OrderItems")
sold_quan = crsr.fetchone()
print(f'\n10. Всього біло продано товарів: {sold_quan}')

crsr.execute("SELECT count(prod_name) FROM Products WHERE prod_price > 4")
prod_name_mt4 = crsr.fetchone()
print(f'\n11. Кількіть найменувань товару, ціна якого становить більше 4: {prod_name_mt4}')

crsr.execute("SELECT prod_name, prod_price FROM Products ORDER BY prod_price DESC")
prod_name_w3 = crsr.fetchmany(3)
print(f'\n12. Три найдорожчих товара в базі:')
for i in prod_name_w3:
    print(i)
    
crsr.execute("""SELECT cust_name, count(order_num)
                FROM Orders
                LEFT JOIN Customers 
                WHERE Orders.cust_id == Customers.cust_id
                GROUP BY cust_name""")
ord_cust_count = crsr.fetchall()
print(f'\n13. Кількість замовлень по кожному клієнту:')
for i in ord_cust_count:
    print(i)

crsr.execute("""SELECT cust_name, order_num 
                FROM Customers JOIN Orders ON Customers.cust_id == Orders.cust_id
                ORDER BY cust_name""")
cust_orders = crsr.fetchall()
print(f'\n14. Клієнти та номера їх замовлень:')
for i in cust_orders:
    print(i)

crsr.execute("SELECT prod_name FROM Products WHERE prod_price < 5")
prod_name_l5 = crsr.fetchall()
print(f'\n15. Товари, ціна яких менше 5:')
for i in prod_name_l5:
    print(i)

crsr.execute("SELECT prod_name FROM Products WHERE prod_price >= 5")
prod_name_we5 = crsr.fetchall()
print(f'\n    Товари, ціна яких >= 5:')
for i in prod_name_we5:
    print(i)

crsr.execute("""SELECT prod_name FROM Products WHERE prod_price < 5
            UNION ALL
            SELECT prod_name FROM Products WHERE prod_price >= 5""")
union_2 = crsr.fetchall()
print(f"\n    Таблиця в якій об'єднані ці 2 запити:")
for i in union_2:
    print(i)


connection.commit()
connection.close()

1. Версія бази даних SQLite: ('3.33.0',)
   База налічує (5,) таблиць.

2. Назви таблиць наступні:
[('Customers',), ('OrderItems',), ('Orders',), ('Products',), ('Vendors',)]

3. Список cust_id із таблиці Customers:
[('1000000001',), ('1000000002',), ('1000000003',), ('1000000004',), ('1000000005',)]

4. Таблиця Customers виглядає наступним чином:
('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)

5. Список клієнтів, відсортованих від Z до A:
('Village Toys',)
('The Toy

#### row_factory
Ігоре, я спробувала використати row_factory, щоб відформувати результат, але мені вдалося це зробити тільки з кортежами з 1 елементу. Чи є ще якісь варіанти?

In [3]:
import sqlite3

tysql = '/Users/Home/Desktop/Python/MainAcademy/tysql.sqlite' 
connection = sqlite3.connect(tysql)
connection.row_factory = sqlite3.Row
crsr = connection.cursor()


crsr.execute("SELECT count(*) from sqlite_master as tables WHERE type='table'")
q_tables = crsr.fetchone()
for i in q_tables:
    print(f'1. База налічує {i} таблиць.')

crsr.execute("SELECT sum(quantity) FROM OrderItems")
sold_quan = crsr.fetchone()
for i in sold_quan:
    print(f'\n10. Всього біло продано товарів: {i}')

crsr.execute("SELECT count(prod_name) FROM Products WHERE prod_price > 4")
prod_name_mt4 = crsr.fetchone()
for i in prod_name_mt4:
    print(f'\n11. Кількіть найменувань товару, ціна якого становить більше 4: {i}')

connection.close()

1. База налічує 5 таблиць.

10. Всього біло продано товарів: 1430

11. Кількіть найменувань товару, ціна якого становить більше 4: 6


## Task 4

Зареєструйтеся на сайті  
http://www.sql-ex.ru/  

_**Виконайте завдання із блоку SELECT  
з 1 по 30 завдання**_  
_Код запитів вставляйте сюди_

### №1
_Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd_

> **SELECT model, speed, hd FROM PC WHERE price < 500**

### №2
_Найдите производителей принтеров. Вывести: maker_
> **SELECT DISTINCT maker FROM Product WHERE type = 'Printer'**

### №3
_Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол._
> **SELECT model, ram, screen FROM Laptop WHERE price > 1000**

### №4
_Найдите все записи таблицы Printer для цветных принтеров._
> **SELECT * FROM Printer WHERE color = 'y'**

### №5
_Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол._
> **SELECT model, speed, hd FROM PC WHERE price < 600 AND (cd = '12x' OR cd = '24x')**

### №6
_Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость._
> **SELECT DISTINCT Product.maker, Laptop.speed   
FROM Laptop JOIN Product  
ON Product.model = Laptop.model  
WHERE hd >= 10  
ORDER BY maker, speed**

### №7
_Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква)._
> **SELECT * FROM (
 SELECT model, price FROM PC
 UNION
 SELECT model, price FROM Laptop
 UNION
 SELECT model, price FROM Printer
 ) AS a
WHERE a.model IN 
(SELECT model FROM Product WHERE maker = 'B')**

### №8
_Найдите производителя, выпускающего ПК, но не ПК-блокноты._
> **SELECT DISTINCT maker 
FROM Product 
WHERE type = 'PC'
EXCEPT
SELECT maker FROM Product
WHERE type = 'Laptop'**

### №9
_Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker_
> **SELECT DISTINCT maker as Maker
FROM Product
JOIN PC  
ON Product.model = PC.model  
WHERE speed >= 450**

### №10
_Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price_
> **SELECT model, price
FROM Printer
WHERE price IN  
(SELECT MAX(price) as price FROM printer)**

### №11
_Найдите среднюю скорость ПК._
> **SELECT AVG(speed) as Avg_speed FROM PC**

### №12
_Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол._
> **SELECT AVG(speed) as Avg_speed FROM Laptop WHERE price > 1000**

### №13
_Найдите среднюю скорость ПК, выпущенных производителем A._
> **SELECT AVG(speed) as Avg_speed FROM PC 
JOIN Product ON PC.model = Product.model 
WHERE maker = 'A'**

### №14
_Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий._
> **SELECT Classes.class, Ships.name, country
FROM Classes 
JOIN Ships ON Classes.class = Ships.class
WHERE numGuns >= 10**

### №15
_Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD_
> **SELECT hd FROM PC
GROUP BY hd
HAVING COUNT(model) >=2**

### №16
_Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM._
> **SELECT DISTINCT one.model, sec.model, one.speed, one.ram
FROM PC one JOIN 
(SELECT speed, ram
FROM PC
GROUP BY speed, ram
HAVING COUNT(speed) >= 2 AND COUNT(ram) >= 2) 
S ON one.speed = S.speed AND one.ram = S.ram 
JOIN PC sec ON sec.speed = S.speed AND sec.ram = S.ram AND 
sec.model < one.model
ORDER BY one.model, sec.model**

### №17
_Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК.
Вывести: type, model, speed_
> **SELECT DISTINCT Product.type, Laptop.model, Laptop.speed FROM PC, Laptop, Product
WHERE type = 'Laptop' AND Laptop.speed < ALL (SELECT PC.speed FROM PC)**

### №18
_Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price_
> **SELECT maker, price
FROM Product JOIN Printer ON
Product.model = Printer.model
WHERE color = 'y'
AND price IN 
(SELECT MIN(price) FROM Printer WHERE color = 'y')
GROUP BY maker, price**

### №19
_Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов.
Вывести: maker, средний размер экрана._
> **SELECT maker, AVG(screen) as Avg_screen
FROM Product JOIN Laptop
ON Product.model = Laptop.model
GROUP BY maker**

### №20
_Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК._
> **SELECT maker, COUNT(type) AS Count_Model  FROM Product
WHERE type = 'PC' 
GROUP BY maker
HAVING COUNT(type) >= 3**

### №21
_Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC.
Вывести: maker, максимальная цена._
> **SELECT maker, MAX(price) as Max_price
FROM Product JOIN PC
ON Product.model = PC.model
GROUP BY maker**

### №22
_Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена._
>**SELECT speed, AVG(price) as Avg_price
FROM PC
WHERE speed > 600
GROUP BY speed**

### №23
_Найдите производителей, которые производили бы как ПК
со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц.
Вывести: Maker_
> **SELECT maker FROM Product
JOIN PC ON Product.model = PC.model
WHERE type = 'PC' AND speed >= 750
INTERSECT 
SELECT maker FROM Product 
JOIN Laptop ON Product.model = Laptop.model
WHERE type = 'Laptop' AND speed >= 750**

### №24
_Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции._
> **SELECT DISTINCT model
FROM (SELECT model, price FROM PC
 UNION ALL
 SELECT model, price FROM Laptop
 UNION ALL
 SELECT model, price FROM Printer
 ) DB
WHERE price = 
(SELECT MAX(price) FROM 
(SELECT model, price FROM PC
 UNION ALL
 SELECT model, price FROM Laptop
 UNION ALL
 SELECT model, price FROM Printer
) NT)**

### №25
_Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker_
> **SELECT maker FROM Product
WHERE type = 'printer'
AND maker IN (SELECT maker FROM Product JOIN PC
ON Product.model = PC.model
WHERE ram = (SELECT MIN(ram) FROM PC) 
AND speed = (SELECT MAX(speed) FROM PC WHERE ram = (SELECT MIN(ram) FROM PC)))
GROUP BY maker**

### №26
_Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена._
> **SELECT AVG(price) AS AVG_price FROM (
 SELECT model, price FROM PC
 UNION ALL
 SELECT model, price FROM Laptop
  ) AS a
WHERE a.model IN 
(SELECT model FROM Product WHERE maker = 'A')**

### №27
_Найдите средний размер диска ПК каждого из тех производителей, которые выпускают и принтеры. Вывести: maker, средний размер HD._
> **SELECT maker, AVG(hd) 
FROM Product
JOIN PC ON Product.model = PC.model
WHERE maker IN (SELECT maker FROM Product 
WHERE type = 'printer'
GROUP BY maker)
GROUP BY maker**

### №28
_Используя таблицу Product, определить количество производителей, выпускающих по одной модели._
> **SELECT COUNT(dd.maker)  FROM
(SELECT maker, COUNT(model) AS qty 
FROM Product
GROUP BY maker
HAVING COUNT(model) = 1) dd**

### №29
_В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o._
> **Все, я більше не можу... це капец...**

### №30
_В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка.
Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL)._
> ** **